## preparation

In [31]:
import pandas as pd
import numpy as np
import scipy.optimize
import random

In [7]:
fares = pd.read_csv('Fares.csv', sep = ';')
flights = pd.read_csv('Flights.csv', sep = ';')


In [12]:
history = pd.read_csv('~\Documents\GitHub\Quant_Marketing_UvA\Ass_3\History\Bookings_001.csv', sep = ';')

In [40]:
history = pd.read_csv('~\Documents\GitHub\Quant_Marketing_UvA\Ass_3\History\Bookings_001.csv', sep = ';')
history['id'] = '001'
for i in range(2,101,1):
    if len(str(i)) == 1:
        i = '00' + str(i)
    elif len(str(i)) == 2:
        i = '0' + str(i)
    else:
        i = str(i)
        
        
    hist = pd.read_csv('~\Documents\GitHub\Quant_Marketing_UvA\Ass_3\History\Bookings_' + i + '.csv', sep = ';')
    hist['id'] = i
    history = history.append(hist)
    
    
   

In [215]:
def get_demands(fare_id, history):
    filt_history = history.loc[history.FARE_ID == fare_id,]
    demands = filt_history['id'].value_counts()
    return demands

def get_capacity(flight_id):
    return flights.loc[flights.FLIGHT_ID == flight_id]['CAPACITY']
    
def get_price(fare_id):
    return fares.loc[fares.FARE_ID == fare_id]['PRICE']

def get_flights(fare_id):
    dep_air  = int(fares.loc[fares.FARE_ID == fare_id]['DEPARTURE_AIRPORT'])
    dest_air = int(fares.loc[fares.FARE_ID == fare_id]['DESTINATION_AIRPORT'])
   
    if dep_air == 1 or dest_air == 1:
        return [flights.loc[(flights.DEPARTURE_AIRPORT == dep_air) &
                           (flights.DESTINATION_AIRPORT == dest_air)]['FLIGHT_ID']]
    else:
        return [flights.loc[(flights.DEPARTURE_AIRPORT == dep_air)]['FLIGHT_ID'], 
                flights.loc[(flights.DESTINATION_AIRPORT == dest_air)]['FLIGHT_ID']]




    

In [259]:
flight_fare_df = pd.DataFrame(index=range(flights.shape[0]),columns=range(fares.shape[0]), data = 0)
for i in range(fares.shape[0]):
    used_flights = get_flights(i+1)
    try: 
        flight_fare_df.iloc[int(used_flights[0]) - 1, i] = 1
        flight_fare_df.iloc[int(used_flights[1]) - 1, i] = 1
    except:
        flight_fare_df.iloc[int(used_flights[0]) - 1, i] = 1
        

## optimization

In [83]:
def kvadr_x(x):
    return (x ** 2 - 3 * x + 8) 

In [96]:
cons = ( {'type': 'ineq',
            'fun' : lambda x: np.array([-x, 0])})

In [97]:
x0 = np.array([100])
scipy.optimize.minimize(kvadr_x, x0,  method='SLSQP', constraints = cons)

     fun: 7.999999999999997
     jac: array([-3.])
 message: 'Optimization terminated successfully.'
    nfev: 10
     nit: 3
    njev: 3
  status: 0
 success: True
       x: array([8.8817842e-16])

In [321]:
#problem je v constraints nekde
def profit(x):
    return -(np.array(fares['PRICE']) @ x)

capacity = flights['CAPACITY']
capacity = np.reshape(capacity, (16))
x0 = np.zeros((fares.shape[0], ))

print(capacity.shape)
print(np.reshape(np.array(flight_fare_df) @ x0,(16)).shape)
print(x0.shape)

cons = ({'type' : 'ineq',
         'fun' : lambda x: np.array([capacity, (np.reshape(np.array(flight_fare_df) @ x,(16)))])})
         
scipy.optimize.minimize(profit, x0,  method='SLSQP', constraints = cons)
         



(16,)
(16,)
(144,)


C:\Users\Radim\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:52: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


ValueError: all the input arrays must have same number of dimensions

In [313]:
capacity = flights['CAPACITY']
np.reshape(capacity, (16)).shape
# capacity - np.array(flight_fare_df) @ x0

C:\Users\Radim\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:52: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


(16,)

In [312]:
(np.reshape(np.array(flight_fare_df) @ x0,(16))).shape

(16,)

array([-0.])

In [306]:
(np.array(flight_fare_df) @ np.zeros((fares.shape[0],1))).shape

(16, 1)

19380530